## Team Map

Pre requisitos:
Tabela com colunas: nome do colaborador, data da avaliação de proficiência(primeira avaliação), nível na avaliação de proficiência, data da última avaliação, nível na última avaliação.

In [1]:
# Nomes dos arquivos
NOME_TABELA_INICIAL = 'TeamMap-BakerAndina.xlsx'
NOME_TABELA_TRANSFORMADA = "./Tabela_Transformada_BA.xlsx"

# Nomes das colunas
NOME_COLUNA_NOME = 'Colaborador'
NOME_COLUNA_IDIOMA = 'Curso'
NOME_COLUNA_NIVEL_AVALIACAO = 'Classificação da Última Avaliação'
NOME_COLUNA_NIVEL_PROFICIENCIA = 'Classificação da Primeira Avaliação'
NOME_COLUNA_DATA_PROFICIENCIA = 'Data da Primeira Avaliação'
NOME_COLUNA_DATA_AVALIACAO = 'Data da Última Avaliação'
NOME_COLUNA_META_FINAL = 'Meta Final'
NOME_COLUNA_STATUS = 'Status'

# Valores de filtro
NOME_STATUS_ATIVO_FILTRO = 'Subsídio ativo'

## Imports

In [2]:
!which python

/home/pessoal/Documentos/gitc/jupyter-graphs-angela/myenv/bin/python


In [3]:
#requirement.txt
!pip install pandas
!pip install openpyxl
!pip install streamlit
!pip install plotly
!pip install matplotlib
!pip install dash

In [4]:
import pandas as pd
import plotly.graph_objects as go
import streamlit
import plotly.express as px
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, HTML

# Table transformer
import openpyxl
import numpy as np
import os
import re
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Alignment, Border, Side, Font
from dateutil.relativedelta import relativedelta





# Transformers

In [5]:
# Carregar os dados do arquivo
data = pd.read_excel(NOME_TABELA_INICIAL)

if NOME_COLUNA_NIVEL_AVALIACAO in data.columns:
    print(f"A coluna '{NOME_COLUNA_NIVEL_AVALIACAO}' não foi encontrada no arquivo Excel.")

if not NOME_COLUNA_NIVEL_PROFICIENCIA in data.columns:
    print(f"A coluna '{NOME_COLUNA_NIVEL_PROFICIENCIA}' não foi encontrada no arquivo Excel.")
if not NOME_COLUNA_STATUS in data.columns:
    print(f"A coluna '{NOME_COLUNA_STATUS}' não foi encontrada no arquivo Excel.")
if not NOME_COLUNA_NOME in data.columns:
    print(f"A coluna '{NOME_COLUNA_NOME}' não foi encontrada no arquivo Excel.")
if not NOME_COLUNA_IDIOMA in data.columns:
    print(f"A coluna '{NOME_COLUNA_IDIOMA}' não foi encontrada no arquivo Excel.")
if not NOME_COLUNA_DATA_PROFICIENCIA in data.columns:
    print(f"A coluna '{NOME_COLUNA_DATA_PROFICIENCIA}' não foi encontrada no arquivo Excel.")
if not NOME_COLUNA_DATA_AVALIACAO in data.columns:
    print(f"A coluna '{NOME_COLUNA_DATA_AVALIACAO}' não foi encontrada no arquivo Excel.")
if not NOME_COLUNA_META_FINAL in data.columns:
    print(f"A coluna '{NOME_COLUNA_META_FINAL}' não foi encontrada no arquivo Excel.")


# Verificar e substituir 'Ground zero' por 'Marco zero' nas colunas de classificações
if NOME_COLUNA_NIVEL_AVALIACAO in data.columns:
    data[NOME_COLUNA_NIVEL_AVALIACAO] = data[NOME_COLUNA_NIVEL_AVALIACAO].replace('Ground zero', 'Marco zero')
    data[NOME_COLUNA_NIVEL_AVALIACAO] = data[NOME_COLUNA_NIVEL_AVALIACAO].replace('Marco cero', 'Marco zero')
if NOME_COLUNA_NIVEL_PROFICIENCIA in data.columns:
    data[NOME_COLUNA_NIVEL_PROFICIENCIA] = data[NOME_COLUNA_NIVEL_PROFICIENCIA].replace('Ground zero', 'Marco zero')
    data[NOME_COLUNA_NIVEL_PROFICIENCIA] = data[NOME_COLUNA_NIVEL_PROFICIENCIA].replace('Marco cero', 'Marco zero')

# Verificar se a coluna 'Status' existe antes de filtrar
if NOME_COLUNA_STATUS in data.columns:
    data.rename(columns=lambda x: x.strip(), inplace=True)  # Remover espaços dos nomes das colunas
    if NOME_COLUNA_STATUS in data.columns:
        # Filtrar apenas os usuários com 'Subsídio ativo'
        data = data[data[NOME_COLUNA_STATUS] == NOME_STATUS_ATIVO_FILTRO]
    else:
        print(f"2. A coluna '{NOME_COLUNA_STATUS}' não foi encontrada no arquivo Excel. Continuando sem filtrar por '{NOME_STATUS_ATIVO_FILTRO}'.")
else:
    print(f"1. A coluna '{NOME_COLUNA_STATUS}' não foi encontrada no arquivo Excel. Continuando sem filtrar por '{NOME_STATUS_ATIVO_FILTRO}'.")

# Ordenar os dados pelo nome do colaborador
if NOME_COLUNA_NOME in data.columns:
    data = data.sort_values(by=NOME_COLUNA_NOME)
else:
    raise KeyError(f"A coluna '{NOME_COLUNA_NOME}' não foi encontrada no arquivo Excel.")

# Lista de classificações da régua da União Europeia (modificada)
idiomas = data[NOME_COLUNA_IDIOMA].unique()
tabelas_classificacoes = {}
for idioma in idiomas:
    classificacoes_idioma = (
        data[data[NOME_COLUNA_IDIOMA] == idioma][NOME_COLUNA_NIVEL_AVALIACAO].unique().tolist() +
        data[data[NOME_COLUNA_IDIOMA] == idioma][NOME_COLUNA_NIVEL_PROFICIENCIA].unique().tolist()
    )
    tabelas_classificacoes[idioma] = list(set(classificacoes_idioma))  # Remover duplicatas

# Função para ordenar as classificações
def ordenar_classificacoes(classificacoes):
    def classificacao_key(nivel):
        if isinstance(nivel, str) and nivel.lower() == 'marco zero':
            return (0, 0, 0)
        elif isinstance(nivel, str):
            if '.' not in nivel:
                letra = nivel
                return (ord(letra[0].upper()) - ord('A') + int(letra[1])/10, 0, 0)
            nivel_split = nivel.split('.')
            letra = nivel_split[0]
            plus = 0
            if '+' in letra:
                letra = letra[:-1]
                plus = 0.1
            numero = int(nivel_split[1])
            return (ord(letra[0].upper()) - ord('A') + int(letra[1])/10, plus, numero)
        return (float('inf'), 0, 0)

    return sorted([x for x in classificacoes if pd.notna(x) and x != ""], key=classificacao_key)

# Função para preencher a tabela transformada
def preencher_tabela_transformada(row, classificacoes):
    linha_usuario = {nivel: np.nan for nivel in classificacoes}
    ciclo_origem = row.get(NOME_COLUNA_DATA_PROFICIENCIA, None)
    ciclo_atual = row.get(NOME_COLUNA_DATA_AVALIACAO, None)
    prazo_meta = row.get(NOME_COLUNA_DATA_AVALIACAO, None)

    if pd.notna(ciclo_origem):
        ciclo_origem = pd.to_datetime(ciclo_origem).strftime('%m/%Y')
    if pd.notna(ciclo_atual):
        ciclo_atual = pd.to_datetime(ciclo_atual).strftime('%m/%Y')
    if pd.notna(prazo_meta):
        prazo_meta_dt = pd.to_datetime(prazo_meta)
        prazo_meta = (prazo_meta_dt + relativedelta(years=2)).strftime('%m/%Y')

    ultima_classificacao = row.get(NOME_COLUNA_NIVEL_AVALIACAO, None)
    proficiencia_original = row.get(NOME_COLUNA_NIVEL_PROFICIENCIA, None)
    meta_final = row.get(NOME_COLUNA_META_FINAL, None)

    # Preencher ciclo de origem
    if proficiencia_original and pd.notna(proficiencia_original) and proficiencia_original in classificacoes:
        linha_usuario[proficiencia_original] = ciclo_origem

    # Preencher última classificação
    if ultima_classificacao and pd.notna(ultima_classificacao) and ultima_classificacao in classificacoes:
        linha_usuario[ultima_classificacao] = ciclo_atual

    # Preencher meta final
    if meta_final and pd.notna(meta_final) and meta_final in classificacoes:
        linha_usuario[meta_final] = prazo_meta

    # Colorir o caminho percorrido
    if (ultima_classificacao and proficiencia_original and
        ultima_classificacao in classificacoes and proficiencia_original in classificacoes and
        ciclo_atual):
        indices = [classificacoes.index(ultima_classificacao), classificacoes.index(proficiencia_original)]
        for i in range(min(indices), max(indices) + 1):
            if classificacoes[i] not in [ultima_classificacao, proficiencia_original]:
                linha_usuario[classificacoes[i]] = 'Caminho Percorrido'

    # Adicionar caminho a percorrer entre ciclo_atual e prazo_meta
    if (ultima_classificacao and meta_final and
        ultima_classificacao in classificacoes and meta_final in classificacoes):
        indices = [classificacoes.index(ultima_classificacao), classificacoes.index(meta_final)]
        for i in range(min(indices), max(indices) + 1):
            if classificacoes[i] not in [ultima_classificacao, meta_final, proficiencia_original]:
                linha_usuario[classificacoes[i]] = 'Caminho a Percorrer'

    # Adicionar caminho a percorrer entre ciclo_origem e prazo_meta se ciclo_atual estiver ausente
    if not ciclo_atual or pd.isna(ciclo_atual):
        if (proficiencia_original and meta_final and
            proficiencia_original in classificacoes and meta_final in classificacoes):
            indices = [classificacoes.index(proficiencia_original), classificacoes.index(meta_final)]
            for i in range(min(indices), max(indices) + 1):
                if classificacoes[i] not in [proficiencia_original, meta_final]:
                    linha_usuario[classificacoes[i]] = 'Caminho a Percorrer'

    return pd.Series(linha_usuario)

# Deletar o arquivo existente, se houver
if os.path.exists(NOME_TABELA_TRANSFORMADA):
    os.remove(NOME_TABELA_TRANSFORMADA)

# Criar tabelas separadas por idioma e salvar em Excel
with pd.ExcelWriter(NOME_TABELA_TRANSFORMADA, engine='openpyxl') as writer:
    for idioma in idiomas:
        classificacoes = ordenar_classificacoes(tabelas_classificacoes[idioma])
        data_filtrada = data[data[NOME_COLUNA_IDIOMA] == idioma]
        tabela_transformada = data_filtrada.apply(preencher_tabela_transformada, axis=1, classificacoes=classificacoes)
        usuarios = data_filtrada[NOME_COLUNA_NOME]
        tabela_transformada.index = usuarios
        tabela_transformada.to_excel(writer, sheet_name=idioma)

        # Ajustar tamanhos das células e aplicar cores
        workbook = writer.book
        worksheet = writer.sheets[idioma]

        # Garantir que a planilha esteja visível
        worksheet.sheet_state = 'visible'

        # Ajustar largura das colunas
        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter  # Coluna
            for cell in col:
                try:
                    if cell.value is not None:
                        cell_length = len(str(cell.value))
                        if cell_length > max_length:
                            max_length = cell_length
                except:
                    pass
            adjusted_width = (max_length + 2)
            worksheet.column_dimensions[column].width = adjusted_width

        # Aplicar cores ao cabeçalho e células
        header_fill = PatternFill(start_color='1f2f36', end_color='1f2f36', fill_type='solid')
        cell_fill = PatternFill(start_color='391e70', end_color='391e70', fill_type='solid')
        cell_fill_meta_final = PatternFill(start_color='adc22f', end_color='adc22f', fill_type='solid')
        cell_fill_caminho_a_percorrer = PatternFill(start_color='adc22f', end_color='adc22f', fill_type='solid')
        header_alignment = Alignment(horizontal='center', vertical='center')
        cell_alignment = Alignment(horizontal='center', vertical='center')
        thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                             top=Side(style='thin'), bottom=Side(style='thin'))
        header_font = Font(color='FFFFFF')

        # Colorir cabeçalho
        for cell in worksheet[1]:
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.font = header_font  # Texto branco
            cell.border = thin_border

        # Colorir primeira coluna
        for cell in worksheet['A']:
            if cell.row > 1:
                cell.fill = header_fill
                cell.font = header_font
                cell.alignment = cell_alignment
                cell.border = thin_border

        # Colorir células preenchidas, o caminho percorrido, e aplicar bordas e alinhamento
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row,
                                       min_col=2, max_col=worksheet.max_column):
            aux = 0
            for cell in row:
                if cell.value not in ['Caminho Percorrido', 'Caminho a Percorrer', ""]:
                    aux += 1

            aux = 0
            for cell in row:
                if cell.value == 'Caminho Percorrido':
                    cell.fill = cell_fill
                    cell.value = ""
                elif cell.value == 'Caminho a Percorrer':
                    cell.fill = cell_fill_caminho_a_percorrer
                    cell.value = ""
                    aux = 2
                elif cell.value is not None and cell.value != "":
                    cell.fill = cell_fill

                    aux += 1
                    if aux == 3:
                        cell.fill = cell_fill_meta_final
                    else:
                        cell.font = header_font
                    cell.border = thin_border
                cell.alignment = cell_alignment

        # Fixar primeira linha e primeira coluna
        worksheet.freeze_panes = 'B2'

print("Tabela salva com sucesso!")


A coluna 'Classificação da Última Avaliação' não foi encontrada no arquivo Excel.
A coluna 'Status' não foi encontrada no arquivo Excel.
1. A coluna 'Status' não foi encontrada no arquivo Excel. Continuando sem filtrar por 'Subsídio ativo'.
Tabela salva com sucesso!


# Database

In [6]:
df = pd.read_excel(NOME_TABELA_TRANSFORMADA)
df.fillna("", inplace=True)
df.head(5)

,Colaborador,Marco zero,A1.1,A1.2,A2.1,A2.2,B1.1,B1.2,B2.1,B2.2,B2+.1,B2+.2,C1.1,C1.2,C1.3
0,Aldys Lopez,,,05/2024,,,,,,05/2026,,,,,
1,Andrea Alexandra Suntaxi Soto,,,,,,,,05/2023,,07/2024,,07/2026,,
2,Anny Lozano Cardenas,,,,06/2023,07/2024,,,07/2026,,,,,,
3,Antonio Lopez,,,,,,,02/2023,,04/2024,,04/2026,,,
4,Anyee Pamela Castillo Zambrano,05/2023,,07/2024,,07/2026,,,,,,,,,


In [7]:
data_original = pd.read_excel(NOME_TABELA_INICIAL, sheet_name=0)
transformed_single_language = pd.read_excel(NOME_TABELA_TRANSFORMADA, sheet_name='Inglês')

# TeamMap

## Distribuição de pessoas por nivel de proficiência

In [8]:
color_scale = [
    "#1f77b4",  # Azul escuro
    "#337ebc",
    "#4c8eb3",
    "#6699aa",
    "#80a0a1",
    "#99a799",
    "#b3ae90",
    "#cca588",
    "#e6b07f",
    "#ffba76",
    "#ffb36d",
    "#ffad63",
    "#ffa559",
    "#ff9f4f",
    "#ff9945",
    "#ff933b",
    "#ff8d31",
    "#ff8727",
    "#ff811d",
    "#ff7b13",
    "#ff7509",
    "#ff6f00"   # Vermelho
]


In [9]:
# dash
import dash
from dash import Dash, dcc, html, Input, Output
from dash.dependencies import Input, Output

# Inicializar a aplicação Dash
app = Dash(__name__)
server = app.server  # Para deploy no Heroku ou similar

# Configurar o renderizador para um compatível com interações
pio.renderers.default = "iframe"  

data_original.rename(columns={NOME_COLUNA_NIVEL_PROFICIENCIA: 'Proficiency'}, inplace=True)

# Substituir valores específicos
data_original['Proficiency'] = data_original['Proficiency'].replace(['Ground zero', 'Marco cero'], 'Marco zero')

# Definir a ordem dos níveis do CEFR
niveis_cefr = [
    "Marco zero", "A1.1", "A1.2", "A2.1", "A2.2", 
    "B1.1", "B1.2", "B2.1", "B2.2", "B2+.1", 
    "B2+.2", "C1.1", "C1.2", "C1.3", "C2"
]

# Obter listas únicas de países e departamentos
lista_paises = data_original['País'].unique()
lista_departamentos = data_original['Departamento'].unique()

# --- Layout da Aplicação ---
app.layout = html.Div([
    html.H1("Dashboard de Distribuição de Proficiência, País e Departamento", style={'textAlign': 'center'}),
    
    # Filtros
    html.Div([
        html.Div([
            html.Label("Selecionar País(es):"),
            dcc.Dropdown(
                id='filtro-pais',
                options=[{'label': pais, 'value': pais} for pais in sorted(lista_paises)],
                value=list(lista_paises),  # Selecionar todos por padrão
                multi=True,
                placeholder="Selecione um ou mais países"
            ),
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '0 20px'}),
        
        html.Div([
            html.Label("Selecionar Departamento(s):"),
            dcc.Dropdown(
                id='filtro-departamento',
                options=[{'label': dept, 'value': dept} for dept in sorted(lista_departamentos)],
                value=list(lista_departamentos),  # Selecionar todos por padrão
                multi=True,
                placeholder="Selecione um ou mais departamentos"
            ),
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '0 20px'}),
    ], style={'padding': '20px'}),
    
    # Gráficos
    html.Div([
        dcc.Graph(id='grafico-proficiencia'),
    ], style={'padding': '20px'}),
    
    html.Div([
        dcc.Graph(id='grafico-pais'),
    ], style={'padding': '20px'}),
    
    html.Div([
        dcc.Graph(id='grafico-departamento'),
    ], style={'padding': '20px'}),
])

# --- Callbacks para Atualizar os Gráficos ---
@app.callback(
    [
        Output('grafico-proficiencia', 'figure'),
        Output('grafico-pais', 'figure'),
        Output('grafico-departamento', 'figure')
    ],
    [
        Input('filtro-pais', 'value'),
        Input('filtro-departamento', 'value')
    ]
)
def atualizar_graficos(paises_selecionados, departamentos_selecionados):
    # Filtrar os dados com base nas seleções
    df_filtrado = data_original[
        data_original['País'].isin(paises_selecionados) &
        data_original['Departamento'].isin(departamentos_selecionados)
    ]
    
    # --- Gráfico de Proficiência ---
    contagem_proficiencia = df_filtrado['Proficiency'].value_counts().reindex(niveis_cefr, fill_value=0)
    porcentagem_proficiencia = (contagem_proficiencia / contagem_proficiencia.sum() * 100).round(2).astype(str) + '%'
    
    dados_proficiencia = pd.DataFrame({
        "Nível de Proficiência": niveis_cefr,
        "Contagem": contagem_proficiencia.values,
        "Porcentagem": porcentagem_proficiencia.values
    })
    
    
    fig_proficiencia = px.bar(
        dados_proficiencia,
        x="Nível de Proficiência",
        y="Contagem",
        title="Distribuição dos Níveis de Proficiência",
        labels={"Contagem": "Número de Pessoas", "Nível de Proficiência": "Níveis do CEFR"},
        text="Porcentagem",
        color="Nível de Proficiência",
        color_discrete_sequence=color_scale
    )
    
    fig_proficiencia.update_traces(textposition='outside')
    fig_proficiencia.update_layout(
        xaxis=dict(categoryorder='array', categoryarray=niveis_cefr),
        template="plotly_white",
        title_font=dict(size=20, family='Arial, bold'),
        xaxis_title_font=dict(size=14, family='Arial'),
        yaxis_title_font=dict(size=14, family='Arial'),
    )
    
    # --- Gráfico de País ---
    contagem_pais = df_filtrado['País'].value_counts().sort_values(ascending=False)
    porcentagem_pais = (contagem_pais / contagem_pais.sum() * 100).round(2).astype(str) + '%'
    
    dados_pais = pd.DataFrame({
        "País": contagem_pais.index,
        "Contagem": contagem_pais.values,
        "Porcentagem": porcentagem_pais.values
    })
    
    fig_pais = px.bar(
        dados_pais,
        x="País",
        y="Contagem",
        title="Distribuição por País",
        labels={"Contagem": "Número de Pessoas", "País": "Países"},
        text="Porcentagem",
        color="País",
        color_discrete_sequence=px.colors.qualitative.Plotly
    )
    
    fig_pais.update_traces(textposition='outside')
    fig_pais.update_layout(
        template="plotly_white",
        title_font=dict(size=20, family='Arial, bold'),
        xaxis_title_font=dict(size=14, family='Arial'),
        yaxis_title_font=dict(size=14, family='Arial'),
    )
    
    # --- Gráfico de Departamento ---
    contagem_departamento = df_filtrado['Departamento'].value_counts().sort_values(ascending=False)
    porcentagem_departamento = (contagem_departamento / contagem_departamento.sum() * 100).round(2).astype(str) + '%'
    
    dados_departamento = pd.DataFrame({
        "Departamento": contagem_departamento.index,
        "Contagem": contagem_departamento.values,
        "Porcentagem": porcentagem_departamento.values
    })
    
    fig_departamento = px.bar(
        dados_departamento,
        x="Departamento",
        y="Contagem",
        title="Distribuição por Departamento",
        labels={"Contagem": "Número de Pessoas", "Departamento": "Departamentos"},
        text="Porcentagem",
        color="Departamento",
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    
    fig_departamento.update_traces(textposition='outside')
    fig_departamento.update_layout(
        template="plotly_white",
        title_font=dict(size=20, family='Arial, bold'),
        xaxis_title_font=dict(size=14, family='Arial'),
        yaxis_title_font=dict(size=14, family='Arial'),
    )
    
    return fig_proficiencia, fig_pais, fig_departamento

# Executar a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)

In [10]:
# Definir o renderizador para um compatível com interações
pio.renderers.default = "iframe"  

# Carregar dados

# Renomear a coluna para facilitar o acesso
data_original.rename(columns={NOME_COLUNA_NIVEL_PROFICIENCIA: 'Proficiency'}, inplace=True)

# Substituir 'Ground zero' por 'Marco zero'
data_original['Proficiency'] = data_original['Proficiency'].replace('Ground zero', 'Marco zero')

# Definir a ordem dos níveis do CEFR
niveis_cefr = [
    "Marco zero", "A1.1", "A1.2", "A2.1", "A2.2", 
    "B1.1", "B1.2", "B2.1", "B2.2", "B2+.1", 
    "B2+.2", "C1.1", "C1.2", "C1.3", "C2"
]

# Contar ocorrências de cada nível do CEFR
contagem_proficiencia = data_original['Proficiency'].value_counts().reindex(niveis_cefr, fill_value=0)

# Calcular porcentagens e adicionar o símbolo %
porcentagem_proficiencia = (contagem_proficiencia / contagem_proficiencia.sum() * 100).round(2)

# Preparar os dados para plotagem
dados_proficiencia = pd.DataFrame({
    "Nível de Proficiência": niveis_cefr,
    "Contagem": contagem_proficiencia.values,
    "Porcentagem": porcentagem_proficiencia.values
})

# Criar um gráfico de barras com uma escala de cores discreta, mostrando porcentagem no eixo Y e número de pessoas como texto acima das barras
fig = px.bar(
    dados_proficiencia,
    x="Nível de Proficiência",
    y="Porcentagem",
    title="Distribuição dos Níveis de Proficiência",
    labels={"Porcentagem": "Porcentagem (%)", "Nível de Proficiência": "Níveis do CEFR"},
    text="Contagem",  # Mostrar a contagem de pessoas diretamente em cada barra
    color="Nível de Proficiência",  # Adicionar diferenciação de cor
    color_discrete_sequence=color_scale  # Usar uma escala de cores qualitativa
)

# Melhorar o layout e a estética
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis=dict(categoryorder='array', categoryarray=niveis_cefr),
    template="plotly_white",  # Tema claro para maior destaque
    title_font=dict(size=20, family='Arial, bold'),
    xaxis_title_font=dict(size=14, family='Arial'),
    yaxis_title_font=dict(size=14, family='Arial'),
)

# Mostrar o gráfico
fig.show()


## Mapeamento individual

In [11]:
# Definir o locale para português
try:
    locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')  # Para sistemas baseados em Unix
except:
    try:
        locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
    except:
        print("Locale português não disponível no sistema. Os meses podem aparecer em inglês.")

# Nome da planilha (sheet) no arquivo Excel transformado
sheet_name = 'Inglês'  # Substitua pelo nome correto da planilha
COLABORADOR = "Nome do colaborador"

# Ler os dados transformados
transformed_single_language = pd.read_excel(NOME_TABELA_TRANSFORMADA, sheet_name=sheet_name)

# Verificar se a coluna 'Nome do colaborador' existe
if NOME_TABELA_NOME not in transformed_single_language.columns:
    raise KeyError(f"A coluna '{COLABORADOR}' não está presente no DataFrame")

# Extrair as colunas relevantes
cef_columns = ['MZ', 'A1', 'A1.1', 'A1.2', 'A2', 'A2.1', 'A2.2',
               'B1', 'B1.1', 'B1.2', 'B2', 'B2.1', 'B2.2', 'B2+',
               'B2+.1', 'B2+.2', 'C1', 'C1.1', 'C1.2', 'C2']
# Manter apenas as colunas existentes no DataFrame
existing_cef_columns = [col for col in cef_columns if col in transformed_single_language.columns]
transformed_single_language = transformed_single_language[[COLABORADOR] + existing_cef_columns]

# Lista completa de colaboradores
all_colaboradores = transformed_single_language[COLABORADOR].unique()

# Extrair a coluna 'Nome do colaborador' e as datas
colaboradores = transformed_single_language[[COLABORADOR]]
dates = transformed_single_language[existing_cef_columns]

# Transformar os dados para o formato longo
long_data = pd.melt(
    pd.concat([colaboradores, dates], axis=1),
    id_vars=[COLABORADOR],
    var_name='CEFR Level',
    value_name='Date'
)

# Remover linhas onde 'Date' é NaN
long_data_non_null = long_data.dropna(subset=['Date'])

# Ordenar os dados por 'Nome do colaborador' e 'CEFR Level'
long_data_non_null['CEFR Level'] = pd.Categorical(long_data_non_null['CEFR Level'], categories=existing_cef_columns, ordered=True)
long_data_non_null = long_data_non_null.sort_values(by=[COLABORADOR, 'CEFR Level'])

# Função para definir as cores dos pontos e linhas
def mark_points(group):
    group = group.reset_index(drop=True)
    n_points = len(group)
    
    # Inicializar cores
    group['Point Color'] = '#391e70'  # Cor padrão: roxo
    group['Line Color'] = '#391e70'   # Cor padrão: roxo

    if n_points >= 1:
        # Primeiro ponto é sempre roxo
        group.at[0, 'Point Color'] = '#391e70'  # Roxo

    if n_points == 2:
        # Se tiver 2 pontos, o segundo é verde
        group.at[1, 'Point Color'] = '#adc22f'  # Verde
    elif n_points >= 3:
        # Se tiver 3 ou mais pontos, os últimos 2 são verdes
        group.at[n_points - 2, 'Point Color'] = '#adc22f'  # Verde
        group.at[n_points - 1, 'Point Color'] = '#adc22f'  # Verde

    # Definir as cores das linhas
    for idx in range(n_points - 1):
        # Se ambos os pontos são roxos, a linha é roxa
        if group.at[idx, 'Point Color'] == '#391e70' and group.at[idx + 1, 'Point Color'] == '#391e70':
            group.at[idx, 'Line Color'] = '#391e70'  # Roxo
        else:
            # Se pelo menos um dos pontos é verde, a linha é verde
            group.at[idx, 'Line Color'] = '#adc22f'  # Verde

    return group

# Aplicar a função a cada 'Nome do colaborador'
long_data_non_null = long_data_non_null.groupby(COLABORADOR, group_keys=False).apply(mark_points)

# Processar a coluna 'Date' para mostrar apenas mês e ano em português
def process_date(value):
    if isinstance(value, str):
        # Remover 'MF' ou 'Meta Final' se presente
        value = value.replace('MF', '').replace('Meta Final', '').strip()
        # Tentar converter para datetime
        try:
            date = pd.to_datetime(value, errors='coerce', dayfirst=True)
            if pd.isnull(date):
                return value  # Retorna o valor original se não for uma data válida
        except:
            return value
    elif isinstance(value, pd.Timestamp):
        date = value
    else:
        return str(value)
    # Retornar mês e ano em português
    return date.strftime('%b/%Y')  # Formato: 'Jan/2022' em português

# Aplicar a função 'process_date' à coluna 'Date'
long_data_non_null['Date_Text'] = long_data_non_null['Date'].apply(process_date)

# Criar a figura
fig = go.Figure()

# Adicionar traços para cada 'Nome do colaborador'
for name in all_colaboradores:
    group = long_data_non_null[long_data_non_null[COLABORADOR] == name]
    if group.empty:
        # Se o colaborador não tem dados, adicionar um traço vazio
        fig.add_trace(
            go.Scatter(
                x=[None],
                y=[name],
                mode='markers',
                marker=dict(size=8, color='rgba(0,0,0,0)'),  # Marcador transparente
                showlegend=False,
                hoverinfo='none'
            )
        )
    else:
        group = group.sort_values('CEFR Level')
        x_values = group['CEFR Level']
        y_values = [name] * len(group)
        text_values = group['Date_Text']
        marker_colors = group['Point Color'].tolist()
        line_colors = group['Line Color'].tolist()

        # Adicionar pontos e linhas
        for i in range(len(group) - 1):
            fig.add_trace(
                go.Scatter(
                    x=x_values.iloc[i:i+2],
                    y=y_values[i:i+2],
                    mode='lines+markers+text',
                    marker=dict(size=8, color=marker_colors[i:i+2]),
                    line=dict(width=2, color=line_colors[i]),
                    text=text_values.iloc[i:i+2],
                    textposition="top center",
                    hoverinfo='text',
                    showlegend=False,
                )
            )
        # Adicionar o último ponto
        fig.add_trace(
            go.Scatter(
                x=[x_values.iloc[-1]],
                y=[y_values[-1]],
                mode='markers+text',
                marker=dict(size=8, color=marker_colors[-1]),
                text=[text_values.iloc[-1]],
                textposition="top center",
                hoverinfo='text',
                showlegend=False,
            )
        )

# Definir a ordem das categorias no eixo x
fig.update_xaxes(
    type='category',
    categoryorder='array',
    categoryarray=existing_cef_columns,
    title_text='Nível CEFR',
    showgrid=True,
    side='top'
)

# Definir o eixo y com todos os colaboradores, mesmo sem dados
fig.update_yaxes(
    type='category',
    categoryorder='array',
    categoryarray=sorted(all_colaboradores),
    title_text='Colaboradores',
    showgrid=False,
    automargin=True,
    autorange='reversed',
    tickfont=dict(size=12),
)

# Ajustar a altura da figura
height_per_collaborator = 20  # Ajuste este valor para aumentar ou diminuir o espaçamento
fig_height = 600

# Atualizar o layout da figura
fig.update_layout(
    title='Progresso dos Colaboradores nos Níveis CEFR',
    template='plotly_white',
    height=fig_height,
    width=1200,
    margin=dict(l=50, r=50, t=50, b=50),
)

# Mostrar a figura
fig.show()


Locale português não disponível no sistema. Os meses podem aparecer em inglês.


NameError: name 'NOME_TABELA_NOME' is not defined

## Distribuição de Colaboradores por Nível CEFR

In [ ]:
import ipywidgets as widgets
display(HTML("""
    <style>
        .output_scroll { height: auto !important; max-height: none !important; }
    </style>
"""))

# Step 2: Load the Excel File
df = pd.read_excel(NOME_TABELA_INICIAL)

# Step 3: Rename Columns for Consistency
df.rename(columns={
    NOME_COLUNA_NOME: 'Colaborador',
    NOME_COLUNA_NIVEL_PROFICIENCIA: 'Classificação'
}, inplace=True)

# Step 4: Define the Grouping Function
def group_level(level):
    """
    Groups the CEFR levels into broader categories.

    Parameters:
    - level (str): The CEFR level of a collaborator.

    Returns:
    - str: The grouped level category.
    """
    if isinstance(level, str):
        if level.startswith('A') or level.startswith('M') or level.startswith('G'):
            return 'Nível limitado'
        elif level.startswith('B'):
            if '+' in level:
                return 'Nível sólido'
            return 'Nível independente'
        elif level.startswith('C'):
            return 'Nível competente'
    return None  # Exclude 'Outro' and any non-matching entries

# Step 5: Apply the Grouping Function
df['Grupo Nível'] = df['Classificação'].apply(group_level)

# Step 6: Filter Out 'Outro' and Any Non-Matching Entries
df_filtered = df.dropna(subset=['Grupo Nível']).copy()

# Step 7: Get Unique 'Idioma' Values
idiomas = df_filtered[NOME_COLUNA_IDIOMA].dropna().unique()
idiomas_sorted = sorted(idiomas)  # Optional: sort the languages alphabetically

# Step 8: Define the Function to Create Sunburst Chart for a Given 'Idioma'
def create_sunburst(selected_idioma):
    """
    Creates a Sunburst chart for the selected 'Idioma'.

    Parameters:
    - selected_idioma (str): The selected language.

    Returns:
    - Plotly Figure: The Sunburst chart.
    """
    # Filter data for the selected 'Idioma'
    df_idioma = df_filtered[df_filtered[NOME_COLUNA_IDIOMA] == selected_idioma].copy()

    # Step 9: Count the Number of Collaborators in Each CEFR Level
    cefr_counts = df_idioma['Classificação'].value_counts().reset_index()
    cefr_counts.columns = ['Classificação', 'Count']

    # Step 10: Assign 'Grupo Nível' to Each CEFR Level
    cefr_counts['Grupo Nível'] = cefr_counts['Classificação'].apply(group_level)

    # Ensure no 'Outro' is present
    cefr_counts = cefr_counts.dropna(subset=['Grupo Nível'])

    # Calculate percentages
    total_count = cefr_counts['Count'].sum()
    cefr_counts['Percentage'] = (cefr_counts['Count'] / total_count) * 100

    # Step 11: Create the Sunburst Chart
    fig = px.sunburst(
        cefr_counts,
        path=['Grupo Nível', 'Classificação'],
        values='Count',
        color='Grupo Nível',
        color_discrete_map={
            'Nível limitado': 'lightblue',
            'Nível independente': 'lightgreen',
            'Nível competente': 'lightcoral',
            'Nível sólido': 'lightgray'
        },
        title=f'Distribuição de Colaboradores por Nível CEFR - Idioma: {selected_idioma} - Proficiência',
        hover_data={'Count': True, 'Percentage': ':.3f'},  # Add percentage to hover data
        labels={'Count': 'Número de Colaboradores', 'Percentage': 'Porcentagem'}
    )

    # Step 12: Update Layout for Better Appearance
    fig.update_layout(
        margin=dict(t=50, l=25, r=25, b=25)
    )

    return fig

# Step 13: Create an Interactive Dropdown Widget
idioma_dropdown = widgets.Dropdown(
    options=idiomas_sorted,
    value=idiomas_sorted[0],  # Set the default value to the first language
    description=NOME_COLUNA_IDIOMA,
    disabled=False,
)

# Step 14: Define the Function to Update the Plot Based on Dropdown Selection
def update_plot(selected_idioma):
    fig = create_sunburst(selected_idioma)
    fig.show()

# Step 15: Use `interactive_output` to Link the Dropdown with the Plot Update Function
out = widgets.interactive_output(update_plot, {'selected_idioma': idioma_dropdown})

# Step 16: Display the Dropdown and the Output Plot
display(idioma_dropdown, out)

In [ ]:
display(HTML("""
    <style>
        .output_scroll { height: auto !important; max-height: none !important; }
    </style>
"""))

df = pd.read_excel(NOME_TABELA_INICIAL)

# Renomear a coluna de 'Nome' para 'Colaborador' (ajuste conforme necessário)
df.rename(columns={NOME_COLUNA_NOME: 'Colaborador'}, inplace=True)

# Função para agrupar níveis CEFR
def group_level(level):
    """
    Agrupa os níveis CEFR em categorias mais amplas.
    """
    if isinstance(level, str):
        if level.startswith('A') or level.startswith('M') or level.startswith('G'):
            return 'Nível limitado'
        elif level.startswith('B'):
            if '+' in level:
                return 'Nível sólido'
            return 'Nível independente'
        elif level.startswith('C'):
            return 'Nível competente'
    return None  # Excluir 'Outro' e entradas não correspondentes

# Obter listas únicas de países, departamentos e idiomas
lista_paises = df['País'].dropna().unique()
lista_departamentos = df['Departamento'].dropna().unique()
idiomas = df[NOME_COLUNA_IDIOMA].dropna().unique()
idiomas_sorted = sorted(idiomas)  # Opcional: ordenar alfabeticamente

# Widget para selecionar o tipo de avaliação
avaliacao_dropdown = widgets.Dropdown(
    options=['Avaliação de Proficiência', 'Avaliação de Acompanhamento'],
    value='Avaliação de Proficiência',
    description='Tipo de Avaliação:',
    disabled=False,
)

# Widget para selecionar País(es) usando SelectMultiple
pais_select = widgets.SelectMultiple(
    options=sorted(lista_paises),
    value=list(lista_paises),  # Selecionar todos por padrão
    description='País(es):',
    disabled=False,
    style={'description_width': 'initial'},
)

# Widget para selecionar Departamento(s) usando SelectMultiple
departamento_select = widgets.SelectMultiple(
    options=sorted(lista_departamentos),
    value=list(lista_departamentos),  # Selecionar todos por padrão
    description='Departamento(s):',
    disabled=False,
    style={'description_width': 'initial'},
)

# Widget para selecionar Idioma
idioma_dropdown = widgets.Dropdown(
    options=idiomas_sorted,
    value=idiomas_sorted[0],  # Valor padrão
    description='Idioma:',
    disabled=False,
)

def create_sunburst(avaliacao_tipo, pais_selecionados, departamento_selecionados, idioma_selecionado):
    """
    Cria um gráfico Sunburst com base nos filtros selecionados.
    """
    df_copy = df.copy()
    
    # Renomear a coluna de Classificação com base no tipo de avaliação
    if avaliacao_tipo == 'Avaliação de Proficiência':
        NOME_COLUNA_CLASSIFICACAO = NOME_COLUNA_NIVEL_PROFICIENCIA  # Substitua pelo nome real
    else:
        NOME_COLUNA_CLASSIFICACAO = NOME_COLUNA_NIVEL_AVALIACAO

    if NOME_COLUNA_CLASSIFICACAO not in df_copy.columns:
        raise ValueError(f"A coluna '{NOME_COLUNA_CLASSIFICACAO}' não existe no DataFrame.")
    df_copy[NOME_COLUNA_CLASSIFICACAO] = df_copy[NOME_COLUNA_CLASSIFICACAO].replace(['Ground zero', 'Marco cero'], 'Marco zero')
    
    
    df_copy.rename(columns={NOME_COLUNA_CLASSIFICACAO: 'Classificação'}, inplace=True)
    
    # Aplicar a função de agrupamento
    df_copy['Grupo Nível'] = df_copy['Classificação'].apply(group_level)
    
    # Filtrar dados com base nos filtros de País e Departamento
    df_filtered = df_copy[
        df_copy['País'].isin(pais_selecionados) &
        df_copy['Departamento'].isin(departamento_selecionados)
    ].copy()
    
    # Filtrar por Idioma
    df_idioma = df_filtered[df_filtered[NOME_COLUNA_IDIOMA] == idioma_selecionado].copy()
    
    # Excluir entradas sem grupo de nível
    df_idioma = df_idioma.dropna(subset=['Grupo Nível'])
    
    # Contar o número de colaboradores por Classificação
    cefr_counts = df_idioma['Classificação'].value_counts().reset_index()
    cefr_counts.columns = ['Classificação', 'Count']
    
    # Reaplicar a função de agrupamento para garantir consistência
    cefr_counts['Grupo Nível'] = cefr_counts['Classificação'].apply(group_level)
    
    # Calcular porcentagens
    total_count = cefr_counts['Count'].sum()
    if total_count > 0:
        cefr_counts['Percentage'] = (cefr_counts['Count'] / total_count) * 100
    else:
        cefr_counts['Percentage'] = 0
    
    # Definir a ordem dos grupos de nível (opcional)
    ordem_grupos = ['Nível limitado', 'Nível independente', 'Nível sólido', 'Nível competente']
    cefr_counts['Grupo Nível'] = pd.Categorical(cefr_counts['Grupo Nível'], categories=ordem_grupos, ordered=True)
    
    # Criar o gráfico Sunburst
    fig = px.sunburst(
        cefr_counts,
        path=['Grupo Nível', 'Classificação'],
        values='Count',
        color='Grupo Nível',
        color_discrete_map={
            'Nível limitado': 'lightblue',
            'Nível independente': 'lightgreen',
            'Nível sólido': 'lightgray',
            'Nível competente': 'lightcoral'
        },
        title=f'Distribuição de Colaboradores por Nível CEFR - Idioma: {idioma_selecionado} - {avaliacao_tipo}',
        hover_data={'Count': True, 'Percentage': ':.2f'},  # Adicionar porcentagem aos dados de hover
        labels={'Count': 'Número de Colaboradores', 'Percentage': 'Porcentagem'}
    )
    
    # Atualizar o layout para melhor aparência
    fig.update_layout(
        margin=dict(t=50, l=25, r=25, b=25)
    )
    
    return fig

def update_plot(avaliacao_tipo, pais_selecionados, departamento_selecionados, idioma_selecionado):
    try:
        fig = create_sunburst(avaliacao_tipo, pais_selecionados, departamento_selecionados, idioma_selecionado)
        fig.show()
    except ValueError as e:
        print(f"Erro: {e}")

# Definir os inputs para a função de atualização
inputs = {
    'avaliacao_tipo': avaliacao_dropdown,
    'pais_selecionados': pais_select,
    'departamento_selecionados': departamento_select,
    'idioma_selecionado': idioma_dropdown
}

# Criar a saída interativa
out = widgets.interactive_output(update_plot, inputs)

# Exibir os widgets e a saída
display(avaliacao_dropdown, pais_select, departamento_select, idioma_dropdown, out)